In [265]:
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,RidgeCV,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE



# Testing the RNN model on the testing data set

In [282]:
#'### Read the data. Filling up missing values with 0
df=pd.read_csv("weather_data/ikermit.csv.gz",compression='gzip')
df["date"] = [datetime.fromisoformat(dt[:-1]) for dt in df.date]
feature_cols = ['WindMax', 'WindGustAlwaysPast1h', 'WindMin', 'WindMinPast1h', 'TempDew', 'Pressure', 'PressureAtSea', 'WindDirPast1h', 'TempDry', 'Weather', 'TempMeanPast1h', 'TempMaxPast1h', 'TempMinPast1h', 'Humidity', 'HumidityPast1h', 'Visibility']

print(f"Using {len(feature_cols)} features: {feature_cols}")
target_col=["WindSpeed"]

Using 16 features: ['WindMax', 'WindGustAlwaysPast1h', 'WindMin', 'WindMinPast1h', 'TempDew', 'Pressure', 'PressureAtSea', 'WindDirPast1h', 'TempDry', 'Weather', 'TempMeanPast1h', 'TempMaxPast1h', 'TempMinPast1h', 'Humidity', 'HumidityPast1h', 'Visibility']


In [283]:
#data splitting again, since I will test the model using the test data set

In [284]:
beg_date = datetime(2021,8,1)
end_date = datetime(2021,12,1)
df = df[(df["date"] > beg_date ) & (df["date"] < end_date)]
print(f"Shape after filtering year 2020: {df.shape}")
df.fillna(0,inplace=True)
df.sort_values(by=["date"],inplace=True)

df_train_full, df_test = train_test_split(df, test_size=0.2,random_state=4,shuffle=False)
df_train, df_val = train_test_split(df_train_full, test_size=0.25,random_state=42, shuffle=False)

Shape after filtering year 2020: (17536, 23)


In [285]:
#check a particular date
this_date = datetime(2021,11,7) 

df_test.reset_index(inplace=True)
min_date_test = df_test.date.min()
max_date_test = df_test.date.max()
if (this_date > max_date_test or this_date < min_date_test):
    print(f"ERROR: {this_date} not in the testing period!")
df_test_row = df_test[df_test["date"] == this_date].index[0]


In [286]:
print(f"Range of dates available in testing period: {min_date_test} - {max_date_test}")
print(f"Row where {this_date} is found: {df_test_row}")

Range of dates available in testing period: 2021-11-06 14:20:00 - 2021-11-30 23:50:00
Row where 2021-11-07 00:00:00 is found: 58


### Model setup, using the training data

In [239]:
# splitting and Data normalization
#def setup_models(df_train,df_test):
X_train = df_train[feature_cols]
y_train = df_train[target_col].values
X_test=df_test[feature_cols]

y_test = df_test[target_col].values
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [287]:
#X_test

In [241]:
# Load the model
import pickle
with open("rnn_model.bin","rb") as f:
    model =pickle.load(f)

In [242]:
model

In [243]:
X_test.shape

(3508, 16)

In [244]:
def split_sequence(sequence1, sequence2, n_steps ,sel_row):
    X, y = list(), list()
    print(f"X {len(sequence1)}")
    print(f"y {len(sequence2)}")
    if len(sequence1) != len(sequence2):
        raise RuntimeError
    for i in range(len(sequence1)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence1)-1:
            break
        # gather input and output parts of the pattern
        #seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        seq_x = sequence1[i:end_ix]
        seq_y = sequence2[i:end_ix]
        X.append(seq_x)
        y.append(seq_y)
    X_out=np.array(X)[sel_row] #only print the row I need
    #print (X_out[58].shape)
    #print(len(X_out))

    #return np.array(X), np.array(y)
    return X_out,np.array(y)

In [245]:
#now trying to test model...
print(X_test.shape)
print(y_test.shape)

(3508, 16)
(3508, 1)


In [246]:
X2_test,y2 = split_sequence(X_test,y_test,16,df_test_row)

X 3508
y 3508


In [247]:
X2_test.shape

(16, 16)

In [248]:
#X2_test

In [249]:
X2_test = np.array([X2_test])

In [250]:
X2_test.shape

(1, 16, 16)

In [251]:
# testing = np.array(X_test)
# testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

In [252]:
#type(X_test)

numpy.ndarray

In [253]:
#X_train.shape

In [254]:
y_pred = model.predict(X2_test)

In [255]:
predicted = y_pred[0][0][0]

In [256]:
expected = y2[0][0][0]

In [257]:
print(f"Model prediction: {predicted}")

Model prediction 3.2006850242614746


In [258]:
print(f"Expected: {expected}")

Expected 4.3
